Ben Trevett 의 [Updated Sentiment Analysis](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/2%20-%20Upgraded%20Sentiment%20Analysis.ipynb) 튜토리얼을 한글 데이터셋에 적용해보는 연습이다. 데이터셋은 [네이버 영화 평점 데이터](https://github.com/e9t/nsmc)을 이용한다.

이 튜토리얼에서는 다음 요소들을 활용하여 1번 모델을 개선해보자.

* packed padded sequences
* pre-trained word embeddings
* different RNN architecture
* bidirectional RNN
* multi-layer RNN
* regularization
* a different optimizer

In [1]:
import torch
from torchtext import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# 전처리

`Field`를 지정하자. 우리는 한글 데이터를 다루므로 토크나이저 또한 별도로 지정해야한다. 여기서는 [KoNLPy](https://konlpy-ko.readthedocs.io/ko/v0.4.3/)의 은전한닢 tokenizer를 이용한다. 또한 패딩과 문장의 길이를 추가한다. 

In [2]:
from konlpy.tag import Mecab
mecab = Mecab()

In [3]:
TEXT = data.Field(tokenize=mecab.morphs, include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

데이터를 불러오고 검증 데이터를 추가한다.

In [4]:
fields = {'text': ('text',TEXT), 'label': ('label',LABEL)}
# dictionary 형식은 {csv컬럼명 : (데이터 컬럼명, Field이름)}

In [43]:
train_data, test_data = data.TabularDataset.splits(
                            path = 'data',
                            train = 'train_data.csv',
                            test = 'test_data.csv',
                            format = 'csv',
                            fields = fields,  
)

In [44]:
import random

train_data, valid_data = train_data.split(random_state=random.seed(SEED))

다음으로 단어 벡터는 전처리된 단어 벡터를 받자. 원 튜토리얼에선 `glove.100d`를 쓰지만 이건 한글을 지원하지 않으므로, 여기선 한글을 지원하는 `fasttext.simple.300d` 를 사용하겠다. 그리고 사전훈련된 단어집에 없는 단어는 0으로 처리하는 걸 방지하기 위해 `unk_init = torch.Tensor.normal_` 옵션을 준다.

In [45]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data,
                max_size = MAX_VOCAB_SIZE,
                vectors = 'fasttext.simple.300d',
                unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [46]:
len(TEXT.vocab)

25002

In [47]:
LABEL.vocab.stoi

defaultdict(None, {'0': 0, '1': 1})

In [48]:
vars(train_data.examples[0])

{'text': ['야한', '장면', '기다리', '는', '것', '도', '곤욕', '이', '군'], 'label': '0'}

In [49]:
for i in range(len(train_data)):
    if len(train_data.examples[i].text)==0:print(i)

데이터 생성자를 만드는데, 길이에 따라 정렬하도록 `sort_within_batch = True` 옵션을 넣어주라고 원 튜토리얼에 되어 있다. 그러나 한글 데이터에선 오류가 발생해서 아래와 같이 `sort_key = lambda x: len(x.text)` 문장을 먼저 넣어줘야 오류없이 작동한다.

In [55]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    #sort_within_batch = True,
    sort_key = lambda x: len(x.text),
    sort_within_batch = True,
    device = device)

In [85]:
next(iter(train_iterator)).text

(tensor([[2284,   41, 9129,  ...,  580,  851, 2105],
         [4076,  260,    4,  ...,   14,   16, 7864],
         [  11,    5, 2224,  ...,  255,  231,   30],
         ...,
         [1129,  950,    7,  ..., 2719,  178,  427],
         [  81,   37, 2736,  ...,   89,    2,    2],
         [ 716,    5,   26,  ...,    1,    1,    1]], device='cuda:0'),
 tensor([19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
         19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
         19, 19, 19, 19, 19, 19, 19, 19, 19, 18, 18, 18, 18, 18, 18, 18, 18, 18,
         18, 18, 18, 18, 18, 18, 18, 18, 18, 18], device='cuda:0'))

패딩 제외 길이로 정렬된 (문장, 길이) 순의 데이터로 이루어져있다.

# 모델 생성

Multi-layered bi-directional LSTM 써서 모델을 생성해보자. 그리고 드랍아웃 적용할거고, `nn.utils.rnn.packed_padded_sequence` 써서 패킹/언패킹 처리할 것이다. 

최종적인 `hidden`의 size는 `[num layers * num directions, batch size, hid dim]`이다. 구체적으로 `[forward_layer_0, backward_layer_0, forward_layer_1, backward_layer 1, ..., forward_layer_n, backward_layer n]`의 형태로 출력되는데, 우리는 꼭대기층의 `hidden`만 필요로 하므로 `hidden[-2::]`과 `hidden[-1::]`만 뽑아서 concatenate할 예정이다.

In [11]:
import torch.nn as nn

nn.Embedding의 padding_idx 옵션에 대해 ARABOZA

In [12]:
emb = nn.Embedding(3,5,padding_idx=1)
test = torch.tensor([0,1,2])

In [13]:
emb(test) # padding_idx 에 해당하는 벡터는 0 이다

tensor([[-1.0478,  0.1479, -0.4590, -0.0847,  1.2871],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.1506, -1.0022, -1.3013,  0.3102, -0.6062]],
       grad_fn=<EmbeddingBackward>)

이제 실제 모델을 구현해보자. 아래 함수는 모델의 벡터들 사이즈 계산할 때 썼던 함수이다.

In [14]:
def print_shape(name, data):
    print(f'{name} has shape {data.shape}')

In [57]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim,
                n_layers, bidirectional, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers,
                          bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        # text = [sent_len, batch_size]
        #print_shape('text',text)
        embedded = self.dropout(self.embedding(text))
        # embedded = [sent_len, batch_size, emb_dim]
        #print_shape('embedded', embedded)
        
        # pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        #print_shape('packed_output', packed_output)
        #print_shape('hidden', hidden)
        #print_shape('cell', cell)
        
        # unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        #print_shape('output', output)
        #print_shape('output_lengths', output_lengths)        
        
        # output = [sent_len, batch_size, hi_dim * num_directions]
        # output over padding tokens are zero tensors
        # hidden = [num_layers * num_directions, batch_size, hid_dim]
        # cell = [num_layers * num_directions, batch_size, hid_dim]
        
        # concat the final forward and backward hidden layers
        # and apply dropout
        
        #print_shape('hidden[-2,:,:]', hidden[-2,:,:])
        #print_shape('hidden[-1,:,:]', hidden[-1,:,:])
        #cat = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        #print_shape('cat', cat)

        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        #print_shape('hidden', hidden)
        # hidden = [batch_size, hid_dim * num_directions]
        
        res = self.fc(hidden)
        #print_shape('res', res)
        return res

하이퍼파라미터를 설정하자.

In [58]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300 # fasttext dim과 동일하게 설정
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

In [59]:
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
           N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

파라미터 갯수 세자. 앞 모델에 비해 월등하게 많은 파라미터 수를 가진다.

In [86]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'이 모델은 {count_parameters(model):,} 개의 파라미터를 가지고 있다.')

이 모델은 10,220,857 개의 파라미터를 가지고 있다.


사전 학습된 fasttext모델의 단어 벡터를 embedding 레이어에 복사하여 담도록 하자.

In [61]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 300])


디멘전 체크 해야됨!

In [62]:
model.embedding.weight.data.shape

torch.Size([25002, 300])

weight가 아니라 weight.data 에 덮어씌워야 한다는 걸 명심!

In [87]:
model.embedding.weight.data.copy_(pretrained_embeddings) # copy_ 메서드는 인수를 현재 모델의 웨이트에 복사함

tensor([[ 2.3087, -1.1270,  0.5406,  ...,  0.5114,  0.1870, -0.4820],
        [ 0.1280, -0.1142, -0.6430,  ...,  1.5142, -0.6999, -0.8775],
        [ 0.0569, -0.0520,  0.2733,  ..., -0.0695, -0.1606, -0.0989],
        ...,
        [-0.6196, -0.4183, -0.5221,  ..., -0.0421, -0.9962, -0.0162],
        [ 0.8182,  0.0668,  0.2058,  ...,  1.4316,  0.0802, -0.5805],
        [-0.2967,  1.0032,  0.2669,  ...,  0.5313,  0.2889, -0.4134]],
       device='cuda:0')

여기서 `<unk`와 `<pad>`는 수동으로 0벡터로 만들자.

In [88]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
UNK_IDX, PAD_IDX

(0, 1)

In [89]:
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0569, -0.0520,  0.2733,  ..., -0.0695, -0.1606, -0.0989],
        ...,
        [-0.6196, -0.4183, -0.5221,  ..., -0.0421, -0.9962, -0.0162],
        [ 0.8182,  0.0668,  0.2058,  ...,  1.4316,  0.0802, -0.5805],
        [-0.2967,  1.0032,  0.2669,  ...,  0.5313,  0.2889, -0.4134]],
       device='cuda:0')


`<pad>`는 pad_idx 옵션 때문에 훈련 내내 0으로 남아있지만 `<unk>` 는 학습될 것이다.

# 모델 학습

Adam으로 학습시킬 것

In [66]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

손실 함수는 `binary cross entropy with logits`로 하자. 이 함수는 임의의 실수를 입력으로 받아서 sigmoid 함수를 취해 0과 1 사이의 값으로 변환한 뒤 `label`과의 `binary cross entropy`를 계산한다.

In [67]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

평가를 위해 임의의 실수를 0과 1 두 정수 중 하나로 변환하는 함수를 만들자.

In [68]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds==y).float()
    acc = correct.sum() / len(correct)
    return acc

훈련 함수 만들자. 현재 batch.text 는 (토큰들,문장 길이) 로 구성되어 있으니 분리하자.

In [90]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        text, text_lengths = batch.text
        predictions = model(text, text_lengths).squeeze(1)
        #print_shape('predictions',predictions)
        
        loss = criterion(predictions, batch.label)
        #print_shape('loss',loss)
        
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

평가를 위한 함수는 그래디언트 업데이트를 하지 않아야 하므로 `with torch.no_grad():` 구문으로 감싸도록 한다. 또한 드랍아웃을 평가 때는 적용하지 않아야 하므로 `model.eval()` 을 넣어주어야 한다.

In [70]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.text
            predictions = model(text, text_lengths).squeeze(1)

            loss = criterion(predictions, batch.label)#.squeeze(0))
            acc = binary_accuracy(predictions, batch.label)#.squeeze(0))

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

에폭마다 걸린 훈련시간을 측정하는 함수를 만든다.

In [71]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

이제 훈련시키자.

In [72]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
        
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 26s
	Train Loss: 0.447 | Train Acc: 78.64%
	 Val. Loss: 0.381 |  Val. Acc: 83.56%
Epoch: 02 | Epoch Time: 0m 26s
	Train Loss: 0.354 | Train Acc: 84.33%
	 Val. Loss: 0.337 |  Val. Acc: 85.77%
Epoch: 03 | Epoch Time: 0m 26s
	Train Loss: 0.314 | Train Acc: 86.49%
	 Val. Loss: 0.322 |  Val. Acc: 86.67%
Epoch: 04 | Epoch Time: 0m 26s
	Train Loss: 0.289 | Train Acc: 87.71%
	 Val. Loss: 0.327 |  Val. Acc: 86.88%
Epoch: 05 | Epoch Time: 0m 26s
	Train Loss: 0.270 | Train Acc: 88.64%
	 Val. Loss: 0.310 |  Val. Acc: 87.43%


앞선 모델의 50%와는 비교할 수 없을만큼 훌륭한 결과를 얻었다.

테스트셋에서의 결과는?

In [73]:
model.load_state_dict(torch.load('tut2-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.313 | Test Acc: 87.25%


나쁘지 않다. 더 훈련시키면 어떻게 되는지 확인해보자.

In [74]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
        
    print(f'Epoch: {epoch+6:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 06 | Epoch Time: 0m 26s
	Train Loss: 0.255 | Train Acc: 89.41%
	 Val. Loss: 0.330 |  Val. Acc: 87.51%
Epoch: 07 | Epoch Time: 0m 26s
	Train Loss: 0.241 | Train Acc: 90.07%
	 Val. Loss: 0.314 |  Val. Acc: 87.66%
Epoch: 08 | Epoch Time: 0m 26s
	Train Loss: 0.229 | Train Acc: 90.53%
	 Val. Loss: 0.330 |  Val. Acc: 87.63%
Epoch: 09 | Epoch Time: 0m 26s
	Train Loss: 0.220 | Train Acc: 90.97%
	 Val. Loss: 0.339 |  Val. Acc: 87.64%
Epoch: 10 | Epoch Time: 0m 26s
	Train Loss: 0.213 | Train Acc: 91.27%
	 Val. Loss: 0.344 |  Val. Acc: 87.47%


이 정도면 이 모델로서는 한계에 봉착했다고 봐도 될 것 같다.

In [75]:
model.load_state_dict(torch.load('tut2-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.313 | Test Acc: 87.25%


# 사용자 데이터 사용

영화 평가 데이터 직접 넣어보자.

다음 기능을 하는 `predict_sentiment` 함수를 만들자.

* sets the model to evaluation mode
* tokenizes the sentence, i.e. splits it from a raw string into a list of tokens
* indexes the tokens by converting them into their integer representation from our vocabulary
* gets the length of our sequence
* converts the indexes, which are a Python list into a PyTorch tensor
* add a batch dimension by unsqueezeing
* converts the length into a tensor
* squashes the output prediction from a real number between 0 and 1 with the `sigmoid` function
* converts the tensor holding a single value into an integer with the item() method

In [77]:
from konlpy.tag import Mecab
mecab = Mecab()

In [80]:
def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok for tok in mecab.morphs(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1) # 배치 
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [81]:
predict_sentiment(model, "이 영화 진짜 재밌었다!!")

0.9646887183189392

In [82]:
predict_sentiment(model, "영화관에서 이걸 본 내가 바보다. 내 돈 돌려줘!!!")

0.007432916201651096

In [83]:
predict_sentiment(model, "이 영화 감독 밥은 먹고 다니냐? 이런 영화 만들고 잠이 와?")

0.02806989662349224

In [84]:
predict_sentiment(model, "내 인생 영화 등극. 주인공한테 너무 몰입해서 시간 가는 줄도 몰랐다...")

0.9872590899467468

이 정도면 나쁘지 않군!